In [299]:
import pandas as pd
import numpy as np
import os
import re
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from dateutil import parser

RAW_DATA_PATH = r"../data/raw"
PROCESSED_DATA_PATH = r"../data/processed"

In [300]:
plt.style.use("seaborn-v0_8-whitegrid")
sns.set_palette("Set2")
plt.rcParams["figure.figsize"] = (12, 8)
plt.rcParams["font.size"] = 12

pd.set_option("display.max_columns", None)

In [301]:
covid_data = pd.read_csv(os.path.join(RAW_DATA_PATH, "covid_19_data.csv"))

Kiểm tra lại bảng dữ liệu

In [302]:
covid_data.head(5)

,SNo,ObservationDate,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered
0,1,01/22/2020,Anhui,Mainland China,1/22/2020 17:00,1.0,0.0,0.0
1,2,01/22/2020,Beijing,Mainland China,1/22/2020 17:00,14.0,0.0,0.0
2,3,01/22/2020,Chongqing,Mainland China,1/22/2020 17:00,6.0,0.0,0.0
3,4,01/22/2020,Fujian,Mainland China,1/22/2020 17:00,1.0,0.0,0.0
4,5,01/22/2020,Gansu,Mainland China,1/22/2020 17:00,0.0,0.0,0.0


In [303]:
covid_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 306429 entries, 0 to 306428
Data columns (total 8 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   SNo              306429 non-null  int64  
 1   ObservationDate  306429 non-null  object 
 2   Province/State   228326 non-null  object 
 3   Country/Region   306429 non-null  object 
 4   Last Update      306429 non-null  object 
 5   Confirmed        306429 non-null  float64
 6   Deaths           306429 non-null  float64
 7   Recovered        306429 non-null  float64
dtypes: float64(3), int64(1), object(4)
memory usage: 18.7+ MB


Với các cột Confirmed, Deaths, Recovered. Ta có thể chuyển sang kiểu dữ liệu int cho phù hợp

In [304]:
covid_data["Confirmed"] = covid_data["Confirmed"].astype(int)
covid_data["Deaths"] = covid_data["Deaths"].astype(int)
covid_data["Recovered"] = covid_data["Recovered"].astype(int)

Strip các cột có dữ liệu dạng str để tránh bị lỗi dư dấu cách 2 phía 

In [305]:
covid_data["Country/Region"] = covid_data["Country/Region"].apply(lambda x: x.strip() if isinstance(x, str) else x)
covid_data["Province/State"] = covid_data["Province/State"].apply(lambda x: x.strip() if isinstance(x, str) else x)

***Làm sạch cột ObservationDate và Last Update.***  

Vấn đề: Dữ liệu trong cột ObservationDate và Last Update là kiểu string, các giá trị không có sự thống nhất về định dạng để chuyển đổi sang datetime bằng phương pháp thủ công.  
Giải quyết: Dùng hàm parser.parse() từ thư viện dateutil.  
- Hàm parse() tự động chuyển đổi các giá trị có kiểu string về datetime nếu string đủ thông tin về thời gian.  
- Các giá trị không thể chuyển đổi sẽ gán dưới dạng pd.NaT (Not A Time) để kiểm tra.  

In [306]:
#Hàm chuyển đổi string về datetime
def safe_parse(date_str): 
    try:
        # Sử dụng parser của dateutil để chuyển đổi
        return parser.parse(date_str, dayfirst=True)  # dayfirst=True ưu tiên định dạng DD/MM/YYYY
    except (ValueError, TypeError):
        # Nếu không chuyển đổi được, trả về NaT (Not a Time)
        return pd.NaT

In [307]:
#Chuyển đổi 2 cột ObservationDate và Last Update về datetime
covid_data['ObservationDate'] = covid_data['ObservationDate'].apply(safe_parse)
covid_data['Last Update'] = covid_data['Last Update'].apply(safe_parse)

In [308]:
#Kiểm tra có giá trị nào không chuyển đổi thành công hay không
print(covid_data['ObservationDate'].isna().any())
print(covid_data['Last Update'].isna().any())

False
False


Kết quả: Không có dữ liệu lỗi, tất cả đã được chuyển thành công

#### Kiểm tra các giá trị null trong dữ liệu và fill các giá trị null

***Kiểm tra từng cột trong bảng có giá trị null hay không***

In [309]:
covid_data.isnull().sum()

SNo                    0
ObservationDate        0
Province/State     78103
Country/Region         0
Last Update            0
Confirmed              0
Deaths                 0
Recovered              0
dtype: int64

In [318]:
covid_data.describe()

,SNo,ObservationDate,Last Update,Confirmed,Deaths,Recovered
count,306429.000000,306429,306429,3.064290e+05,306429.000000,3.064290e+05
mean,153215.000000,2020-11-09 04:39:31.122837760,2021-02-01 02:00:10.825701376,8.567091e+04,2036.403268,5.042029e+04
min,1.000000,2020-01-02 00:00:00,2020-01-02 01:52:00,-3.028440e+05,-178.000000,-8.544050e+05
25%,76608.000000,2020-07-11 00:00:00,2021-02-04 15:13:53,1.042000e+03,13.000000,1.100000e+01
50%,153215.000000,2020-11-06 00:00:00,2021-02-04 15:13:53,1.037500e+04,192.000000,1.751000e+03
75%,229822.000000,2021-03-04 00:00:00,2021-02-04 15:13:53,5.075200e+04,1322.000000,2.027000e+04
max,306429.000000,2021-12-05 00:00:00,2021-12-05 04:21:29,5.863138e+06,112385.000000,6.399531e+06
std,88458.577156,NaN,NaN,2.775516e+05,6410.938048,2.015124e+05


Ta thấy chỉ có cột Province/State có giá trị null với tỉ lệ khoảng 25% => nên tìm cách fill các dữ liệu cột Province/State

In [310]:
missed_province_count_df = covid_data[covid_data["Province/State"].isnull()].loc[:,["SNo","Country/Region"]].groupby("Country/Region").count().reset_index()
missed_province_count_df.rename({"SNo": "null_count"},axis=1, inplace=True)
province_count_df = covid_data.loc[:,["SNo","Country/Region"]].groupby("Country/Region").count().reset_index()
province_count_df.rename({"SNo": "total_count"}, axis=1, inplace=True)

In [320]:
joined_df = pd.merge(missed_province_count_df, province_count_df, on="Country/Region", how="inner")
joined_df["missed_rate"] = (joined_df["null_count"] / joined_df["total_count"]) * 100
joined_df.sort_values("missed_rate", ascending=True, inplace=True)
joined_df.head(25)

,Country/Region,null_count,total_count,missed_rate
10,Australia,3,3788,0.079197
165,Russia,122,30251,0.403292
41,Colombia,84,12503,0.671839
131,Mexico,83,12282,0.675786
101,Japan,127,18059,0.703250
26,Brazil,85,10229,0.830971
155,Peru,83,9625,0.862338
208,Ukraine,90,9891,0.909918
92,India,132,13182,1.001365
40,Chile,78,6453,1.208740


=> Có số lượng lớn quốc gia có tỉ lệ null cột Province/State là 100%. Cần tìm ra 1 giá trị thay thế ví dụ như No Information, Unknown,...

Kiểm tra các giá trị của Province/State của một quốc gia có tỉ lệ null vừa phải để tìm cách fill các giá trị null

In [312]:
covid_data[covid_data["Country/Region"] == "Japan"]["Province/State"].unique()

array([nan, 'Aichi', 'Akita', 'Aomori', 'Chiba', 'Ehime', 'Fukui',
       'Fukuoka', 'Fukushima', 'Gifu', 'Gunma', 'Hiroshima', 'Hokkaido',
       'Hyogo', 'Ibaraki', 'Ishikawa', 'Kagawa', 'Kagoshima', 'Kanagawa',
       'Kochi', 'Kumamoto', 'Kyoto', 'Mie', 'Miyagi', 'Miyazaki',
       'Nagano', 'Nagasaki', 'Nara', 'Niigata', 'Oita', 'Okayama',
       'Okinawa', 'Osaka', 'Port Quarantine', 'Saga', 'Saitama', 'Shiga',
       'Shimane', 'Shizuoka', 'Tochigi', 'Tokushima', 'Tokyo', 'Tottori',
       'Toyama', 'Unknown', 'Wakayama', 'Yamagata', 'Yamaguchi',
       'Yamanashi', 'Iwate'], dtype=object)

Ta thấy ở Nhật Bản có sử dụng Unknown có các giá trị bị mất\
=> Ta có thể sử dụng Unkwown cho các giá trị null

In [313]:
covid_data.fillna({"Province/State": "Unknown"}, inplace=True)